In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import stopwords

Importing Data - PhiUSIIL Phishing URL Dataset <br>
https://archive.ics.uci.edu/dataset/967/phiusiil+phishing+url+dataset

In [3]:
phi_us = pd.read_csv("Data\PhiUSIIL_Dataset.csv")
phi_us.head()

,Unnamed: 0,FILENAME,URL,URLLength,Domain,DomainLength,IsDomainIP,TLD,URLSimilarityIndex,CharContinuationRate,...,NoOfCSS,NoOfJS,NoOfSelfRef,NoOfEmptyRef,NoOfExternalRef,label,TLDProbability,3gramembeddings,4gramembeddings,5gramembeddings
0,0,521848.txt,https://www.southbankmosaics.com,31,www.southbankmosaics.com,24,0,com,100.0,1.000000,...,20,28,119,0,124,1,0.438823,"[[-0.843953013420105, 0.9524714946746826, 0.62...","[[-1.3191587924957275, 0.22187286615371704, -2...","[[0.44881772994995117, -0.025055943056941032, ..."
1,1,31372.txt,https://www.uni-mainz.de,23,www.uni-mainz.de,16,0,de,100.0,0.666667,...,9,8,39,0,217,1,0.019120,"[[-0.843953013420105, 0.9524714946746826, 0.62...","[[-1.3191587924957275, 0.22187286615371704, -2...","[[0.44881772994995117, -0.025055943056941032, ..."
2,2,597387.txt,https://www.voicefmradio.co.uk,29,www.voicefmradio.co.uk,22,0,uk,100.0,0.866667,...,2,7,42,2,5,1,0.014087,"[[-0.843953013420105, 0.9524714946746826, 0.62...","[[-1.3191587924957275, 0.22187286615371704, -2...","[[0.44881772994995117, -0.025055943056941032, ..."
3,3,554095.txt,https://www.sfnmjournal.com,26,www.sfnmjournal.com,19,0,com,100.0,1.000000,...,27,15,22,1,31,1,0.438823,"[[-0.843953013420105, 0.9524714946746826, 0.62...","[[-1.3191587924957275, 0.22187286615371704, -2...","[[0.44881772994995117, -0.025055943056941032, ..."
4,4,151578.txt,https://www.rewildingargentina.org,33,www.rewildingargentina.org,26,0,org,100.0,1.000000,...,15,34,72,1,85,1,0.038436,"[[-0.843953013420105, 0.9524714946746826, 0.62...","[[-1.3191587924957275, 0.22187286615371704, -2...","[[0.44881772994995117, -0.025055943056941032, ..."


### MetaData of the Dataset

In [5]:
phi_us.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 235795 entries, 0 to 235794
Data columns (total 61 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   Unnamed: 0                  235795 non-null  int64  
 1   FILENAME                    235795 non-null  object 
 2   URL                         235795 non-null  object 
 3   URLLength                   235795 non-null  int64  
 4   Domain                      235795 non-null  object 
 5   DomainLength                235795 non-null  int64  
 6   IsDomainIP                  235795 non-null  int64  
 7   TLD                         235795 non-null  object 
 8   URLSimilarityIndex          235795 non-null  float64
 9   CharContinuationRate        235795 non-null  float64
 10  TLDLegitimateProb           235795 non-null  float64
 11  URLCharProb                 235795 non-null  float64
 12  TLDLength                   235795 non-null  int64  
 13  NoOfSubDomain 

In [6]:
phi_us.describe()

,Unnamed: 0,URLLength,DomainLength,IsDomainIP,URLSimilarityIndex,CharContinuationRate,TLDLegitimateProb,URLCharProb,TLDLength,NoOfSubDomain,...,Crypto,HasCopyrightInfo,NoOfImage,NoOfCSS,NoOfJS,NoOfSelfRef,NoOfEmptyRef,NoOfExternalRef,label,TLDProbability
count,235795.000000,235795.000000,235795.000000,235795.000000,235795.000000,235795.000000,235795.000000,235795.000000,235795.000000,235795.000000,...,235795.000000,235795.000000,235795.000000,235795.000000,235795.000000,235795.000000,235795.000000,235795.000000,235795.000000,235168.000000
mean,117897.000000,34.573095,21.470396,0.002706,78.430778,0.845508,0.260423,0.055747,2.764456,1.164758,...,0.023474,0.486775,26.075689,6.333111,10.522305,65.071113,2.377629,49.262516,0.571895,0.218001
std,68068.297699,41.314153,9.150793,0.051946,28.976055,0.216632,0.251628,0.010587,0.599739,0.600969,...,0.151403,0.499826,79.411815,74.866296,22.312192,176.687539,17.641097,161.027430,0.494805,0.211897
min,0.000000,13.000000,4.000000,0.000000,0.155574,0.000000,0.000000,0.001083,2.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000001
25%,58948.500000,23.000000,16.000000,0.000000,57.024793,0.680000,0.005977,0.050747,2.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.007700
50%,117897.000000,27.000000,20.000000,0.000000,100.000000,1.000000,0.079963,0.057970,3.000000,1.000000,...,0.000000,0.000000,8.000000,2.000000,6.000000,12.000000,0.000000,10.000000,1.000000,0.050966
75%,176845.500000,34.000000,24.000000,0.000000,100.000000,1.000000,0.522907,0.062875,3.000000,1.000000,...,0.000000,1.000000,29.000000,8.000000,15.000000,88.000000,1.000000,57.000000,1.000000,0.438823
max,235794.000000,6097.000000,110.000000,1.000000,100.000000,1.000000,0.522907,0.090824,13.000000,10.000000,...,1.000000,1.000000,8956.000000,35820.000000,6957.000000,27397.000000,4887.000000,27516.000000,1.000000,0.438823


In [7]:
phi_us['label'].value_counts()

1    134850
0    100945
Name: label, dtype: int64

### Feature Extraction: Frequency Distribution of trusted Domains

We retrieved the TLD's of the TRANCO's top 1 million sites and extracted a new feature called TLD Probabilities. This is obtained by the following formula:

TLD Probability = (Occurence of TLD) / (Total occurances of all TLDs)

In [12]:
trusted_domains = pd.read_csv('Data\dom_trusted.csv', header=None)
trusted_domains.columns = ['Index', 'Domain']

In [13]:
# Extraction of Top Level Domains from the Domain names
def extract_tld(domain):
    return domain.split('.')[-1] if '.' in domain else None
trusted_domains['TLD'] = trusted_domains.iloc[:, 1].apply(extract_tld)

In [14]:
# This dictionary counts the occurances of each TLD and store them as key value pairs
tld_counts = {}

for tld in trusted_domains['TLD']:
    if tld in tld_counts:
        tld_counts[tld] += 1
    else:
        tld_counts[tld] = 1

In [18]:
tld_ratios = {tld: count / len(trusted_domains) for tld, count in tld_counts.items()}

In [19]:
phi_us['TLDProbability'] = phi_us['TLD'].map(tld_ratios)

Feature Extraction - <br> <br>
Extracting N-GRAM Word2Vec Character Embeddings
A Word2Vec model is trained on the URLs using character-level N-grams. N-gram embeddings (e.g., 3-grams, 4-grams, and 5-grams) are generated for each URL and used as features for modeling. 

In [23]:
import pandas as pd
from gensim.models import Word2Vec

def generate_ngrams(sequence):
    n = 3
    return [sequence[i:i+n] for i in range(len(sequence) - n + 1)]

phi_us['3grams'] = phi_us['URL'].apply(generate_ngrams)

threegrams = list(phi_us['3grams'])
model = Word2Vec(threegrams, vector_size=20, window=4, min_count=1, sg=1, epochs=30)

def get_embedding(token):
    try:
        return model.wv[token].tolist()
    except KeyError:
        return [0.0] * 100
    
phi_us['3gramembeddings'] = phi_us['3grams'].apply(lambda tokens: [get_embedding(token) for token in tokens])

print(phi_us[['3grams', '3gramembeddings']])

                                                   3grams  \
0       [htt, ttp, tps, ps:, s:/, ://, //w, /ww, www, ...   
1       [htt, ttp, tps, ps:, s:/, ://, //w, /ww, www, ...   
2       [htt, ttp, tps, ps:, s:/, ://, //w, /ww, www, ...   
3       [htt, ttp, tps, ps:, s:/, ://, //w, /ww, www, ...   
4       [htt, ttp, tps, ps:, s:/, ://, //w, /ww, www, ...   
...                                                   ...   
235790  [htt, ttp, tps, ps:, s:/, ://, //w, /ww, www, ...   
235791  [htt, ttp, tps, ps:, s:/, ://, //w, /ww, www, ...   
235792  [htt, ttp, tps, ps:, s:/, ://, //w, /ww, www, ...   
235793  [htt, ttp, tps, ps:, s:/, ://, //p, /pa, pat, ...   
235794  [htt, ttp, tps, ps:, s:/, ://, //w, /ww, www, ...   

                                          3gramembeddings  
0       [[-0.843953013420105, 0.9524714946746826, 0.62...  
1       [[-0.843953013420105, 0.9524714946746826, 0.62...  
2       [[-0.843953013420105, 0.9524714946746826, 0.62...  
3       [[-0.84395301342010

In [25]:
import pandas as pd
from gensim.models import Word2Vec

def generate_ngrams(sequence):
    n = 4
    return [sequence[i:i+n] for i in range(len(sequence) - n + 1)]

phi_us['4grams'] = phi_us['URL'].apply(generate_ngrams)

fourgrams = list(phi_us['4grams'])
model = Word2Vec(fourgrams, vector_size=20, window=4, min_count=1, sg=1, epochs=30)

def get_embedding(token):
    try:
        return model.wv[token].tolist()
    except KeyError:
        return [0.0] * 100
    
phi_us['4gramembeddings'] = phi_us['4grams'].apply(lambda tokens: [get_embedding(token) for token in tokens])

print(phi_us[['4grams', '4gramembeddings']])

                                                   4grams  \
0       [http, ttps, tps:, ps:/, s://, ://w, //ww, /ww...   
1       [http, ttps, tps:, ps:/, s://, ://w, //ww, /ww...   
2       [http, ttps, tps:, ps:/, s://, ://w, //ww, /ww...   
3       [http, ttps, tps:, ps:/, s://, ://w, //ww, /ww...   
4       [http, ttps, tps:, ps:/, s://, ://w, //ww, /ww...   
...                                                   ...   
235790  [http, ttps, tps:, ps:/, s://, ://w, //ww, /ww...   
235791  [http, ttps, tps:, ps:/, s://, ://w, //ww, /ww...   
235792  [http, ttps, tps:, ps:/, s://, ://w, //ww, /ww...   
235793  [http, ttps, tps:, ps:/, s://, ://p, //pa, /pa...   
235794  [http, ttps, tps:, ps:/, s://, ://w, //ww, /ww...   

                                          4gramembeddings  
0       [[-1.3191587924957275, 0.22187286615371704, -2...  
1       [[-1.3191587924957275, 0.22187286615371704, -2...  
2       [[-1.3191587924957275, 0.22187286615371704, -2...  
3       [[-1.31915879249572

In [28]:
def generate_ngrams(sequence):
    n = 5
    return [sequence[i:i+n] for i in range(len(sequence) - n + 1)]

phi_us['5grams'] = phi_us['URL'].apply(generate_ngrams)

fivegrams = list(phi_us['5grams'])
model = Word2Vec(fivegrams, vector_size=20, window=4, min_count=1, sg=1, epochs=30)

def get_embedding(token):
    try:
        return model.wv[token].tolist()
    except KeyError:
        return [0.0] * 100
    
phi_us['5gramembeddings'] = phi_us['5grams'].apply(lambda tokens: [get_embedding(token) for token in tokens])

print(phi_us[['5grams', '5gramembeddings']])

                                                   5grams  \
0       [https, ttps:, tps:/, ps://, s://w, ://ww, //w...   
1       [https, ttps:, tps:/, ps://, s://w, ://ww, //w...   
2       [https, ttps:, tps:/, ps://, s://w, ://ww, //w...   
3       [https, ttps:, tps:/, ps://, s://w, ://ww, //w...   
4       [https, ttps:, tps:/, ps://, s://w, ://ww, //w...   
...                                                   ...   
235790  [https, ttps:, tps:/, ps://, s://w, ://ww, //w...   
235791  [https, ttps:, tps:/, ps://, s://w, ://ww, //w...   
235792  [https, ttps:, tps:/, ps://, s://w, ://ww, //w...   
235793  [https, ttps:, tps:/, ps://, s://p, ://pa, //p...   
235794  [https, ttps:, tps:/, ps://, s://w, ://ww, //w...   

                                          5gramembeddings  
0       [[0.44881772994995117, -0.025055943056941032, ...  
1       [[0.44881772994995117, -0.025055943056941032, ...  
2       [[0.44881772994995117, -0.025055943056941032, ...  
3       [[0.448817729949951

In [32]:
phi_us.drop(columns=['3grams','4grams','5grams'], inplace=True)

In [34]:
phi_us.to_csv('Data\PhiUSIIL_Dataset.csv')

Cleaning Step - HTML Labels Cleaning <br><br>
This column consists of text data which needs to be cleaned for further
processing. The cleaning process involved the following steps:
1) Converting Text to Lowercase
2) Removing Extra White space
3) Removing Non-Alphanumeric Characters
4) Removing Stop Words

In [2]:
phi_us = pd.read_csv('Data\PhiUSIIL_Dataset.csv')

In [3]:
stop_words = set(stopwords.words('english'))

In [4]:
def clean_title(title):
    title = title.lower()
    title = re.sub(r'[^a-z0-9\s]', '', title)
    title = re.sub(r'\s+', ' ', title).strip()
    title = ' '.join([word for word in title.split() if word not in stop_words])
    return title

phi_us['HTMLTitle'] = phi_us['Title'].apply(clean_title)